In [1]:
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

# Parallel Functions

In [2]:
class WithExtraArgs(object):
    def __init__(self, func, **args):
        self.func = func
        self.args = args
    def __call__(self, df):
        return self.func(df, **self.args)

def applyParallel(data, func,pool,partition, kwargs):
    data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
    #data_split = np.array_split(data, min(partitions,data.shape[0]))
    data =pool.map(WithExtraArgs(func, **kwargs), data_split)
    #data = pd.concat(pool.map(WithExtraArgs(func, **kwargs), data_split))
    return data

def parallelize(data, func,pool,partition):
    data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
    #data_split = np.array_split(data, partitions)
    data =pool.map(func, data_split)
    return data

In [3]:
cores = cpu_count() #Number of CPU cores on your system
partitions = cores
partitions

28

In [4]:
#!python -c 'import tensorflow as tf; print(tf.__version__)'
#!python -c 'import keras as kr; print(kr.__version__)'

In [5]:
import pickle
f = open("geo_vect_dict.pkl","rb")
geohash_dict = pickle.load(f)
f.close()

In [6]:
import pickle
f = open("geo_dict.pkl","rb")
geo_dict = pickle.load(f)
f.close()

In [7]:
import pickle
f = open("NLP_vect_dict.pkl","rb")
NLP_dict = pickle.load(f)
f.close()

# geohash feature vectors

In [8]:
def onhot_enoceder(train):
    myEncoder = OneHotEncoder(sparse=False)
    myEncoder.fit(train['HOD_cat'].values.reshape(-1, 1))

    onehot_encode = pd.concat([train.reset_index().drop('HOD_cat',1),
                pd.DataFrame(myEncoder.transform(train['HOD_cat'].values.reshape(-1, 1)),
                             columns=['HOD_en0','HOD_en1','HOD_en2','HOD_en3','HOD_en4'])], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [9]:
def one_hot_check(X_res):
    X_res[:,0:10] = np.round(X_res[:,0:10])
        
    X_res[:,20:] = np.round(X_res[:,20:])
    
    return X_res
    

In [10]:
def create_train_set_aug_geo(frame_list,geomap):
    process_name = str(multiprocessing.current_process())
    id = int(process_name.split(',')[0].split('-')[1])
    print("process ",id," started")
    geo_index=[]
    acc_count=[]
    #f_X_train = []
    #f_y_train = []
    print ("process list with length of ",len(frame_list))
    for frame in frame_list:
        #X_train = []
        #y_train = []
        training_set = frame.values
        #geo_vec = geomap[frame.Geohash.iloc[0]]
        geo_code = geo_dict[frame.Geohash.iloc[0]]
        """
        try:
            NLP_code = NLP_dict[frame.Geohash.iloc[0]]
        except:
            NLP_code = np.zeros(100)
        """
        tag = False
        counter=0
        for i in range(8, training_set.shape[0]):
            
            if training_set[i, 1] > 0 :
                tag = True
                counter+=1
                """
                sequence = training_set[i-8:i,4:].flatten()
                #a = np.concatenate((training_set[i-8:i,4:].flatten(),geo_vec),axis=0)
                #a = np.concatenate((a,NLP_code),axis=0)
                #a = np.append(a, geo_code)
                X_train.append(sequence)
                y_train.append(1)
                """
                
            """
            elif random.uniform(0, 1) > 0.98:
            #else:
                sequence = training_set[i-8:i,4:].flatten()
                #a = np.concatenate((training_set[i-8:i,4:].flatten(),geo_vec),axis=0)
                #a = np.concatenate((a,NLP_code),axis=0)
                #a = np.append(a, geo_code)
                X_train.append(sequence)
                y_train.append(0)
            """
        #SMOTE
        if tag == True:
            geo_index.append(geo_code)
            acc_count.append(counter)
        
    #return X_train, y_train
    return geo_index,acc_count

In [11]:
# for logistic regression
def create_sequences(df,geohash_dict):
    #df  = df.head(4000)
    frame_list=[]
    for idx, frame in df.groupby(df.Geohash):
        frame_list.append(frame)
    
    pool = Pool(cores)
    partition = int(np.ceil(float(len(frame_list))/partitions))
    #train_set = applyParallel (frame_list,create_train_set,pool,partition,{'geomap':geohash_dict.copy()})
    train_set = applyParallel (frame_list,create_train_set_aug_geo,pool,partition,{'geomap':geohash_dict.copy()})
    pool.close()
    pool.join()
    code=[]
    count=[]
    for set_ in train_set:
        code.extend(set_[0])
        count.extend(set_[1])
        

   
    return code,count

In [12]:
def train_data(filename):
    df = pd.read_hdf(filename+'.h5',key='set3')
    #display(df.head())
    df_normalize = df.copy()
    train = df_normalize#[df_normalize.TimeStep <= df_normalize.TimeStep.max()*5/6]
    #test = df_normalize[df_normalize.TimeStep > df_normalize.TimeStep.max()*5/6]
    
    
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(train.loc[:,'T-BrokenVehicle':]) 
    scaled_values = scaler.transform(train.loc[:,'T-BrokenVehicle':]) 
    train.loc[:,'T-BrokenVehicle':] = scaled_values
    #scaled_values = scaler.transform(test.loc[:,'T-BrokenVehicle':]) 
    #test.loc[:,'T-BrokenVehicle':] = scaled_values
    #display(test.head())
    
    train = onhot_enoceder(train)
    #test = onhot_enoceder(test)

    #display(train.columns)
    
    code_list,count_list = create_sequences(train,geohash_dict)
    np.save('accident_report/'+filename+'_code',np.array(code_list))
    np.save('accident_report/'+filename+'_count',np.array(count_list))
    #return code_list,count_list
    #print (X_train.shape)
    #X_test, y_test = create_sequences(test,geohash_dict)

    
    #np.save('train_set/X_train_smote_'+filename,X_train)
    #print (X_train.shape)
    #np.save('train_set/y_train_smote_'+filename,y_train)
    #print( y_train.shape)
    """
    np.save('train_set/X_test_'+filename,X_test)
    print (X_test.shape)
    np.save('train_set/y_test_'+filename,y_test)
    print (y_test.shape)
    """

In [14]:
train_data('Atlanta')

/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


process  29  started
process list with length of  3
process  30  started
process list with length of  3
process  31  started
process list with length of  3
process  32  started
process list with length of  3
process  33  started
process list with length of  3
process  34  started
process list with length of  3
process  35  started
process list with length of  3
process list with length of  3
process  36  started
process  37  started
process list with length of  3
process  38  started
process list with length of  3
process  39  started
process list with length of  3
process  40  started
process list with length of  3
process  41  started
process list with length of  3
process  42  started
process list with length of  3
process list with length of  3
process  43  started
process  44  started
process list with length of  3
process  45  started
process list with length of  3
process  46  started
process list with length of  3
process  47  started
process list with length of  3
process  48 

In [15]:
np.load('accident_report/'+'Atlanta'+'_code.npy')

array([483, 481, 176, 933, 484, 498, 499, 496, 497, 502, 503, 500, 501,
       494, 233, 493, 744, 745, 620, 917, 775, 911, 793, 778, 159, 343,
       344, 375, 377, 373, 374, 361, 795, 363, 357, 359, 360, 366, 367,
       368, 470, 469, 468, 467, 101, 473, 472, 471, 465, 464, 488, 492,
       354, 489])

In [16]:
np.load('accident_report/'+'Atlanta'+'_count.npy')

array([  1,   5,   1,  38,  16,  19,  10,  43,  14,  39,   2,  22,   9,
        80,  26, 310,  26,   7,   7,  16, 114,  21,   6,  11,  29,   9,
         1,  30, 200,  50,  33, 416,  23, 166,  25,  36,  47,  35,  45,
        17, 120,   2, 186,   1,   6,  20,  33,   7, 105,  12,  23,  35,
        79,   3])

In [17]:
train_data('Austin')

/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


process  57  started
process list with length of  5
process  58  started
process list with length of  5
process  59  started
process list with length of  5
process  60  started
process list with length of  5
process  61  started
process list with length of  5
process  62  started
process list with length of  5
process  63  started
process list with length of  5
process  64  started
process list with length of  5
process  65  started
process list with length of  5
process  66  started
process list with length of  5
process  67  started
process list with length of  5
process  68  started
process list with length of  5
process  69  started
process  70  started
process list with length of  5
process list with length of  5
process  71  started
process list with length of  5
process list with length of  5
process  72  started
process  73  started
process list with length of  5
process  74  started
process list with length of  5
process  75  started
process list with length of  5
process  76 

In [18]:
train_data('Charlotte')

/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


process  85  started
process list with length of  5
process  86  started
process list with length of  5
process  87  started
process list with length of  5
process  88  started
process list with length of  5
process  89  started
process list with length of  5
process  90  started
process list with length of  5
process  91  started
process list with length of  5
process  92  started
process list with length of  5
process  93  started
process list with length of  5
process  94  started
process list with length of  5
process  95  started
process list with length of  5
process  96  started
process list with length of  5
process  97  started
process list with length of  5
process  98  started
process list with length of  5
process  99  started
process list with length of  5
process  100  started
process list with length of  5
process  101  started
process list with length of  5
process  102  started
process list with length of  5
process  103  started
process list with length of  5
process 

In [19]:
train_data('Dallas')

/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


process  113  started
process list with length of  6
process  114  started
process list with length of  6
process  115  started
process list with length of  6
process  116  started
process list with length of  6
process  117  started
process list with length of  6
process  118  started
process list with length of  6
process  119  started
process list with length of  6
process  120  started
process list with length of  6
process list with length of  6
process  121  started
process  122  started
process list with length of  6
process  123  started
process list with length of  6
process  124  started
process list with length of  6
process  125  started
process list with length of  6
process  126  started
process list with length of  6
process  127  started
process list with length of  6
process  128  started
process list with length of  6
process  129  started
process list with length of  6
process  130  started
process list with length of  6
process  131  started
process list with length

In [20]:
train_data('Houston')

/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


process  141  started
process list with length of  9
process  142  started
process list with length of  9
process  143  started
process list with length of  9
process  144  started
process list with length of  9
process  145  started
process list with length of  9
process  146  started
process list with length of  9
process  147  started
process list with length of  9
process  148  started
process list with length of  9
process  149  started
process list with length of  9
process  150  started
process list with length of  9
process  151  started
process list with length of  9
process  152  started
process list with length of  9
process  153  started
process list with length of  9
process  154  started
process list with length of  9
process  155  started
process list with length of  9
process  156  started
process list with length of  9
process  157  started
process list with length of  9
process  158  started
process list with length of  9
process  159  started
process list with length

In [21]:
train_data('LosAngeles')

/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


process  169  started
process list with length of  6
process  170  started
process list with length of  6
process  171  started
process list with length of  6
process  172  started
process list with length of  6
process  173  started
process list with length of  6
process  174  started
process list with length of  6
process  175  started
process list with length of  6
process  176  started
process list with length of  6
process  177  started
process list with length of  6
process  178  started
process list with length of  6
process  179  started
process list with length of  6
process  180  started
process list with length of  6
process  181  started
process list with length of  6
process  182  started
process list with length of  6
process  183  started
process list with length of  6
process  184  started
process list with length of  6
process  185  started
process list with length of  6
process  186  started
process list with length of  6
process  187  started
process list with length

In [22]:
train_data('Miami')

/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


process  197  started
process list with length of  2
process  198  started
process list with length of  2
process  199  started
process list with length of  2
process  200  started
process list with length of  2
process  201  started
process list with length of  2
process  202  started
process list with length of  2
process  203  started
process list with length of  2
process  204  started
process list with length of  2
process  205  started
process list with length of  2
process  206  started
process list with length of  2
process  207  started
process list with length of  2
process  208  started
process list with length of  2
process  209  started
process list with length of  2
process  210  started
process list with length of  2
process  211  started
process list with length of  2
process  212  started
process list with length of  2
process  213  started
process list with length of  2
process  214  started
process list with length of  2
process  215  started
process list with length

In [ ]:
train_data('all_cities')